# Homework Set 10

**Author**: Chase Coleman

**Date**: 28 April 2016

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from numba import jit, vectorize


%matplotlib inline

## Globalization and Synchronization of Innovation Cycles

In this assignment, we were asked to reproduce some of the figures from "Globalization and Synchronization of Innovation Cycles" by Matsuyama, Sushko, and Gardini. In this paper, the authors use pieces from Judd (1985) and Helpman Krugman (1985) to develop a two country model with trade and innovation. As trade costs fall and intercountry competition increases, the innovation incentives become aligned and countries synchronize their innovation cycles. Additionally, this paper introduces the idea of *coupled oscillators* in the context of an economic model -- They are the first to present a model in which the coupled oscillators arise endogenously within the model.

### Model

In this model, time is discrete with $t = 0, 1, \dots$. There are two countries indexed by $j$ (sometimes $k$) with values 1 or 2. In each of the two countries there is a representative household who inelastically supplies labor $L_j$ at the wage rate $w_{j, t}$. Since the countries will be identical except for in labor supplied then without loss of generality we will say $L_{1} \geq L_{2}$. Households consume a single nontradeable final good which is produced competitive by combining two types of tradeable intermediate inputs using Cobb-Douglas technology

$$Y_{k, t} = C_{k, t} = \left( \frac{X^o_{k, t}}{1 - \alpha} \right)^{1-\alpha} \left( \frac{X_{k, t}}{\alpha} \right)^{\alpha}$$

There are two types of goods above: $X^o_{k, t}$ and $X_{k, t}$. $X^o_{k, t}$ is a homogenous input which can be produced using a linear technology that converts a single unit of labor into a single unit of the homogenous good. This good is freely tradeable, competitively supplied, and homogenous across countries. By choosing the price of this good as numeraire, we know that $w_{j, t} \geq 1$ with $w_{j, t} = 1$ whenever country $j$ produces the homogenous input. $X_{k, t}$ is a composite good that consists of many differentiated goods. Namely,

$$X_{k, t}^{1 - \frac{1}{\sigma}} = \int_{\Omega_t} \left[ x_{k, t}(\nu) \right]^{1 - \frac{1}{\sigma}} d \nu$$

where $x_{k, t}(\nu)$ is the total amount of a differentiated good $\nu \in \Omega_t$ that is produced. $\sigma > 1$ is the direct partial elasticity of substitution between a pair of varieties and $\Omega_t$ is the set of varieties available in period $t$. We can split the varieties into those which are supplied competitively and those supplied monopolistically -- $\Omega_t = \Omega_t^c + \Omega_t^m.

We can derive the demand for differentiated inputs as

$$x_{k, t}(\nu) = \left( \frac{p_{k, t}(\nu)}{P_{k, t}} \right)^{-\sigma} \frac{\alpha w_{k, t} L_k}{P_{k, t}}$$

where $p_{k, t}$ is the price of the variety $\nu$ and $P_{k, t}$ is the price index for differentiated inputs in $k$, $\left[ P_{k, t} \right]^{1 - \sigma} = \int_{\Omega_t} [p_{k, t}]^{1-\sigma} d\nu$. The price of a variety also depends on $k$ because shipping varieties between countries incurs an iceberg trade cost $\tau_{j,k}$. Thus the effective price in country $k$ of a variety $\nu$ produced in country $j$ becomes $p_{k, t}(\nu) = \tau_{j,k} p_{j, t}(\nu)$.

$f$ is amount of labor needed to introduce a new variety

### Law of Motion for Country Specific Varieties

We will work with a normalized measure of our varieties. Namely,

\begin{align*}
  n_{j, t} &= \frac{\theta \sigma f N_{j, t}^c}{\alpha (L_1 + L_2)} \\
  i_{j, t} &= \frac{\theta \sigma f N_{j, t}^m}{\alpha (L_1 + L_2)} \\
  m_{j, t} &= \frac{\theta \sigma f M_{j, t}}{\alpha (L_1 + L_2)} = n_{j, t} + \frac{i_{j, t}}{\theta}
\end{align*}

Given an initial condition, $n_0 = (n_{1, 0}, n_{2, 0}) \in \mathbb{R}_{+}^{2}$, the equilibrium trajectory, $\{ n_t \}_{t=0}^{\infty} = \{ (n_{1, t}, n_{2, t}) \}_{t=0}^{\infty}$, is obtained by iterating the 2D-dynamical system, $n_{t+1} = F(n_t)$ where $F : \mathbb{R}_{+}^{2} \rightarrow \mathbb{R}_{+}^{2}$ given by

\begin{align*}
  F(n_t) &= \begin{cases}
  \big( \delta (\theta s_j(\rho) + (1-\theta) n_{j, t}), \delta (\theta s_j(\rho) + (1-\theta) n_{j, t}) \big) \; &\text{for } n_t \in D_{LL} := \{ (n_1, n_2) \in \mathbb{R}_{+}^{2} | n_j \leq s_j(\rho) \} \\
  \big( \delta n_{1, t}, \delta n_{2, t} \big) \; &\text{for } n_t \in D_{HH} := \{ (n_1, n_2) \in \mathbb{R}_{+}^{2} | n_j \geq h_j(\rho) \} \\
  \big( \delta n_{1, t}, \delta (\theta h_2(n_{1, t}) + (1-\theta) n_{2, t} \big) &\text{for } n_t \in D_{HL} :=  \{ (n_1, n_2) \in \mathbb{R}_{+}^{2} | n_1 \geq s_1(\rho) \wedge n_2 \leq h_2(n_1) \} \\
  \big( \delta (\theta h_1(n_{2, t}) + (1-\theta) n_{1, t}, \delta n_{2, t} \big) &\text{for } n_t \in D_{LH} :=  \{ (n_1, n_2) \in \mathbb{R}_{+}^{2} | n_1 \leq h_1(n_2) \wedge n_2 \geq s_2(\rho) \} \\
  \end{cases}
\end{align*}

where

$$s_1(\rho) = 1 - s_2(\rho) = \min \{ \frac{s_1 - \rho s_2}{1 - \rho}, 1 \}$$

and $h_j(n_k)$ defined implicitly by

$$1 = \frac{s_j}{h_j(n_k) + \rho n_k} + \frac{s_k}{h_j(n_k) + n_k / \rho}$$